# Studying micrograd + Building it from scratch

### engine.py

In [1]:
from math import exp
import numpy as np
import torch

In [2]:
class Value:
    def __init__(self, data, _children=(), _op=""):
        self.data = data
        self.grad = 0.0
        # internal variables
        self._backward = lambda: None
        self._prev = set(_children) # stores previous objects
        self._op = _op # stores the operation
    
    def __add__(self, other): # addition
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, (self, other), '+')
        def _backward():
            self.grad += 1.0 * out.grad
            other.grad += 1.0 * out.grad
        out._backward = _backward
        return out

    def __mul__(self, other): # multiplication
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, (self, other), '*')
        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        out._backward = _backward
        return out
    
    def __pow__(self, other): # power function
        assert isinstance(other, (int, float)), "must be a number not a Value"
        out = Value(self.data ** other, (self,), "**")
        def _backward():
            self.grad += (other * self.data ** (other - 1)) * out.grad
        out._backward = _backward
        return out

    def relu(self): # relu function
        out = Value(0 if self.data < 0 else self.data, (self,), "ReLU")
        def _backward():
            self.grad += (out.data > 0) * out.grad
        out._backward = _backward
        return out
    
    def backward(self): # performs backpropagation to calculate gradients for all the nodes
    
        # topological order
        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        build_topo(self)
        # applies chain rule one var at a time
        self.grad = 1.0
        for node in reversed(topo):
            node._backward()
    
    def __neg__(self): # negative function
        return self * -1

    def __sub__(self, other): # subtraction
        return self + (-other)
    
    def __radd__(self, other): # rev addition
        return self + other
    
    def __rsub__(self, other): # rev subtraction
        return other + (-self)
    
    def __rmul__(self, other): # rev multiplication
        return self * other
    
    def __truediv__(self, other): # true division
        return self * other**-1
    
    def __rtruediv__(self, other): # rev true division
        return other * self**-1
    
    def __repr__(self):
        return f"Value(data={self.data}, grad={self.grad})" # stores the value and it's grad

### test_engine.py

In [3]:
def test_sanity_check():

    x = Value(-4.0)
    z = 2 * x + 2 + x
    q = z.relu() + z * x
    h = (z * z).relu()
    y = h + q + q * x
    y.backward()
    xmg, ymg = x, y

    x = torch.Tensor([-4.0]).double()
    x.requires_grad = True
    z = 2 * x + 2 + x
    q = z.relu() + z * x
    h = (z * z).relu()
    y = h + q + q * x
    y.backward()
    xpt, ypt = x, y

    # forward pass went well
    assert ymg.data == ypt.data.item()
    # backward pass went well
    assert xmg.grad == xpt.grad.item()

    print("test_sanity_check passed!")

def test_more_ops():

    a = Value(-4.0)
    b = Value(2.0)
    c = a + b
    d = a * b + b**3
    c += c + 1
    c += 1 + c + (-a)
    d += d * 2 + (b + a).relu()
    d += 3 * d + (b - a).relu()
    e = c - d
    f = e**2
    g = f / 2.0
    g += 10.0 / f
    g.backward()
    amg, bmg, gmg = a, b, g

    a = torch.Tensor([-4.0]).double()
    b = torch.Tensor([2.0]).double()
    a.requires_grad = True
    b.requires_grad = True
    c = a + b
    d = a * b + b**3
    c = c + c + 1
    c = c + 1 + c + (-a)
    d = d + d * 2 + (b + a).relu()
    d = d + 3 * d + (b - a).relu()
    e = c - d
    f = e**2
    g = f / 2.0
    g = g + 10.0 / f
    g.backward()
    apt, bpt, gpt = a, b, g

    tol = 1e-6
    # forward pass went well
    assert abs(gmg.data - gpt.data.item()) < tol
    # backward pass went well
    assert abs(amg.grad - apt.grad.item()) < tol
    assert abs(bmg.grad - bpt.grad.item()) < tol

    print("test_more_ops passed!")

test_sanity_check()
test_more_ops()

test_sanity_check passed!
test_more_ops passed!
